In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from main_functions import *
from DGD_DP import DGD_DP as dgd_dp
import pickle
import os
%load_ext autoreload
%autoreload 2


In [3]:
with open(os.path.join("data","first_database.pkl"), "rb") as f:
    x,y = pickle.load(f)
n= 100
m = 10
#mélange et séléction des données
X = x[:n]
Y = y[:n]
stack = np.stack([X,Y]).T
np.random.shuffle(stack)
x_shuffle = stack.T[0]
y_shuffle = stack.T[1]
centers_kernel = np.array([x_shuffle[10*i] for i in range(10)])
kernel_center_of_agent = np.array([[0,1],[2,3],[4,5],[6,7],[8,9]])
data_point_of_agent = np.array([np.arange(20*i,20*i+20) for i in range(5)])

In [4]:
#matrices d'adjacence pour dual decomposition et ADMM
A1 = np.ones((5,5)) #fully connected
A2 = np.array([[1,0,1,1,1],[0,1,1,0,1],[1,1,1,1,1],[1,0,1,1,0],[1,1,1,0,1]]) #small world
A3 = np.array([[1,1,0,0,1],[1,1,1,0,0],[0,1,1,1,0],[0,0,1,1,1],[1,0,0,1,1]]) #cycle
A4 = np.array([[1,1,0,0,1],[1,1,1,0,0],[0,1,1,1,0],[0,0,1,1,1],[0,0,0,1,1]]) #line
A5= np.array([[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0],[0,0,0,0,1]]) #non connexe

In [5]:
instance_pb = instance(x_shuffle,centers_kernel,y_shuffle,kernel_center_of_agent,data_point_of_agent,A2,0.5)

In [7]:
solver_non_cooperative = gradient_descent(instance_pb,0.0001,2000,np.ones(10))
true_solution = solver_non_cooperative.solve(verbose=True)
instance_pb.true_solution =  true_solution
print(solver_non_cooperative.curent_solution)

[-0.07506045  0.71943576  0.3676139  -0.24501944  0.7974386  -0.27637492
  0.98171623  0.0710878  -0.54246145 -0.47082757]


In [11]:
solver_DGD = dgd_dp(instance_pb,1e-1, 12,np.ones(10))
solver_DGD.solve(laplacian_scale=1e-2,gamma_init=0.5,verbose=True,package_loss = False, probability_package_loss = 0, decay=0.9)
print(solver_DGD.current_solution)

Iterration : 0
Agent 0 norme du gradient : 1054.3512482763
Agent 1 norme du gradient : 898.9346960327821
Agent 2 norme du gradient : 1149.2763345598437
Agent 3 norme du gradient : 983.4848273412903
Agent 4 norme du gradient : 946.0542781935135
Agent 1 objective : 7557.002698695957
Agent 2 objective : 7557.002698695957
Agent 3 objective : 7557.002698695957
Agent 4 objective : 7557.002698695957
Agent 5 objective : 7557.002698695957
Iterration : 1
Agent 0 norme du gradient : 384098.16448684636
Agent 1 norme du gradient : 279694.50963851163
Agent 2 norme du gradient : 440391.58163799695
Agent 3 norme du gradient : 311977.9407412836
Agent 4 norme du gradient : 302011.8897102537
Agent 1 objective : 914897336.5238891
Agent 2 objective : 673203708.4013481
Agent 3 objective : 1104745182.8256588
Agent 4 objective : 808125572.9869102
Agent 5 objective : 745466359.973995
Iterration : 2
Agent 0 norme du gradient : 70015889.47070777
Agent 1 norme du gradient : 43486637.223685585
Agent 2 norme du gra

In [10]:
print(solver_non_cooperative.curent_solution)

[-0.07506045  0.71943576  0.3676139  -0.24501944  0.7974386  -0.27637492
  0.98171623  0.0710878  -0.54246145 -0.47082757]


In [9]:
solver_DGD = DGD(instance_pb,0.001,1000,np.ones(10))
solver_DGD.solve(verbose=False,package_loss = False, probability_package_loss = 0)
print(solver_DGD.curent_solution)

[[-0.07742128  0.72189477  0.36069501 -0.24925914  0.79989307 -0.28099657
   0.97788428  0.07011467 -0.55118716 -0.48149035]
 [-0.0559573   0.73796744  0.37602481 -0.22720497  0.81504866 -0.25885481
   0.98954904  0.09093174 -0.52881437 -0.46087627]
 [-0.07366843  0.72416332  0.36824015 -0.24498701  0.80206534 -0.27661406
   0.98448976  0.07349897 -0.54540144 -0.47422028]
 [-0.09265097  0.70957512  0.35469354 -0.26449701  0.78821612 -0.29620656
   0.97453517  0.05506746 -0.5653274  -0.49273159]
 [-0.06989083  0.72457082  0.37524822 -0.24049156  0.8021127  -0.27198183
   0.99095133  0.07667317 -0.53937414 -0.46722104]]


In [45]:
1/(10**(0.9))

0.1258925411794167

In [47]:
((11**(0.3))/1)**(1/2)

1.4328769708732303

In [48]:
((1**(0.3))/1)**(1/2)

1.0